# 2048 Solver
This Algorithm is based on the approach as described by the paper 'AI Plays 2048' by Yun Nie, Wenqi Hou and Yicheng An. The Minimax approach to play the game works on the idea of a player(in this case the AI) and the opponent(in case the algorithm which adds the 2 and 4 tiles randomly on the grid).
The Link for the aforementioned paper - https://cs229.stanford.edu/proj2016/report/NieHouAn-AIPlays2048-report.pdf

## Logic
The idea is that for each state of the game, there are atmost 4 possibilities for the next state, if the person moves up, down, left or right. Now after the move we see if there is any change in the board, if not then it is not a valid move. If it is a valid move then the game adds a 2 or 4 tile randomly on the board. Then we create 4 new possibilities for the state of the grid. Now we need to decide which path to follow, which move wil be best at what stage of the game. So we need to define a few things first. Let's say the state of the game be a node, the move or direction be the edge and the final state be the leaf node. Exactly like a tree, we just have to find the right branches to reach the final result of the 2048 leaf. 

So here we want that the numbers are arranged in an 'S' shaped matrix. Such that the numbers add upto each other easily, and larger numbers remain surrounded by largers numbers. To do this we need to add weights to the grid matrix, so we have added the weights in an exponential form, because we are also dealing with exponential numbers. 


We calculate upto 4 states ahead for each confirmed node, which is also called the depth of the tree. But now the calculation becomes long and time-consuming, so we need to prune some branches of the tree. This is done by Alpha - Beta Pruning. The basic idea is that we stop going forward if the current state of the board cannot be improved anymore, by doing any move. At any given node we need to calculate 2^8 or 256 possible states, but we stop calculating if this condition is met. Alpha represents the best score the maximizing player can achieve, while beta represents the best score the the random tile function can achieve. As the algorithm explores the tree, it updates alpha and beta values. If it finds that a certain branch can’t improve the current alpha-beta limits, it "prunes" that branch.

So basically the Minimizer i.e. the random tile function check all the possible empty tiles at a given state and then allocates the worst possible tile in that place. Now if the current worst is still better that the next worst then we go ahead. Same goes with the maximising algorithm, where proceed only if the player moves in the direction which maximises the weighted score of the grid, if it lesser than the current score then we stop moving in that direction. Simple!

In [1]:
# import the Necessary Libraries that will be used in this program

import tkinter as tk
import random
import copy

# define a class for all the functions for the game 2048 and the simulation which runs on it
class G2049:
    def __init__(self, window):
        self.window = window
        self.window.title("2048")
        self.board = [[0] * 4 for _ in range(4)]
        self.score = 0
        self.moves = 0
        self.game_over = False
        self.setup_gui()
        self.add_new_tile()
        self.add_new_tile()
        self.update_gui()
        self.window.bind("<Key>", self.key_pressed)
        self.start_autoplay()

    # ------------------------- GUI Methods -------------------------
    # This is the Graphical User Interface of the game, which is quite similiar to that of the original one. The principle of the game is still the same
    # we can move in 4 directions and if the state of the board changes after a move then in any of the empty tiles, a 2 or 4 tile randomly spawns
    def setup_gui(self):
        self.window.configure(bg="#BBADA0")
        self.header = tk.Frame(self.window, bg="#BBADA0")
        self.header.pack(pady=10)

        # Score and moves display
        self.score_label = tk.Label(self.header, text=f"Score: {self.score}", 
                                   font=("Helvetica", 16, "bold"), bg="#BBADA0", fg="white")
        self.score_label.pack(side="left", padx=20)
        
        self.moves_label = tk.Label(self.header, text=f"Moves: {self.moves}", 
                                   font=("Helvetica", 16, "bold"), bg="#BBADA0", fg="white")
        self.moves_label.pack(side="right", padx=20)

        # Game grid
        self.grid = tk.Frame(self.window, bg="#BBADA0")
        self.grid.pack()
        
        self.cells = []
        for i in range(4):
            row = []
            for j in range(4):
                cell = tk.Label(self.grid, text="", font=("Helvetica", 24, "bold"),
                               width=4, height=2, relief="raised", borderwidth=3,
                               bg="#CDC1B4", fg="#776E65")
                cell.grid(row=i, column=j, padx=5, pady=5)
                row.append(cell)
            self.cells.append(row)
    # Thiis is the colored grid for the game.
    def update_gui(self):
        colors = {
            0: "#9e948a", 2: "#eee4da", 4: "#ede0c8",
            8: "#f2b179", 16: "#f59563", 32: "#f67c5f",
            64: "#f65e3b", 128: "#edcf72", 256: "#edcc61",
            512: "#edc850", 1024: "#edc53f", 2048: "#edc22e"
        }
        for i in range(4):
            for j in range(4):
                value = self.board[i][j]
                self.cells[i][j].config(
                    text=str(value) if value != 0 else "",
                    bg=colors.get(value, "#000000"),
                    fg="#776e65" if value < 8 else "#f9f6f2"
                )
        self.score_label.config(text=f"Score: {self.score}")
        self.moves_label.config(text=f"Moves: {self.moves}")

    # ------------------------- Game Logic -------------------------
    def add_new_tile(self):
        empty_cells = [(i, j) for i in range(4) for j in range(4) if self.board[i][j] == 0]
        if empty_cells:
            i, j = random.choice(empty_cells)
            self.board[i][j] = 2 if random.random() < 0.8 else 4

    def perform_simulated_move(self, board, direction):
        def merge_row(row):
            non_zero = [num for num in row if num != 0]
            merged = []
            i = 0
            while i < len(non_zero):
                if i < len(non_zero)-1 and non_zero[i] == non_zero[i+1]:
                    merged.append(non_zero[i] * 2)
                    i += 2
                else:
                    merged.append(non_zero[i])
                    i += 1
            merged += [0] * (4 - len(merged))
            return merged

        simulated_board = [row.copy() for row in board]

        if direction == "left":
            for i in range(4):
                simulated_board[i] = merge_row(simulated_board[i])
        elif direction == "right":
            for i in range(4):
                simulated_board[i] = merge_row(simulated_board[i][::-1])[::-1]
        elif direction == "up":
            transposed = list(map(list, zip(*simulated_board)))
            for i in range(4):
                transposed[i] = merge_row(transposed[i])
            simulated_board = list(map(list, zip(*transposed)))
        elif direction == "down":
            transposed = list(map(list, zip(*simulated_board)))
            for i in range(4):
                transposed[i] = merge_row(transposed[i][::-1])[::-1]
            simulated_board = list(map(list, zip(*transposed)))

        return simulated_board

    def check_game_over(self, board):
        if any(0 in row for row in board):
            return False
        for i in range(4):
            for j in range(4):
                if (j < 3 and board[i][j] == board[i][j+1]) or \
                   (i < 3 and board[i][j] == board[i+1][j]):
                    return False
        return True

    # ------------------------- AI Logic -------------------------
    # here we have used the Minimax approach to solve the problem.
    def calculate_weighted_score(self, board):
        weights = [
            [2**16, 2**15, 2**14, 2**13],
            [2**9, 2**10, 2**11, 2**12],
            [2**8, 2**7, 2**6, 2**5],
            [2**1, 2**2, 2**3, 2**4]
        ]
        return sum(board[i][j] * weights[i][j] for i in range(4) for j in range(4))

    def maximizer(self, board, depth, alpha, beta):
        if depth == 0 or self.check_game_over(board):
            return self.calculate_weighted_score(board)
        
        max_value = float('-inf')
        for direction in ["left", "right", "up", "down"]:
            new_board = self.perform_simulated_move(board, direction)
            if new_board != board:  # Only evaluate valid moves
                value = self.minimizer(new_board, depth-1, alpha, beta)
                max_value = max(max_value, value)
                alpha = max(alpha, value)
                if beta <= alpha:
                    break  # Beta cutoff
        return max_value

    def minimizer(self, board, depth, alpha, beta):
        if depth == 0 or self.check_game_over(board):
            return self.calculate_weighted_score(board)
        
        min_value = float('inf')
        empty_cells = [(i, j) for i in range(4) for j in range(4) if board[i][j] == 0]
        
        for cell in empty_cells:
            for tile in [2, 4]:
                new_board = [row.copy() for row in board]
                new_board[cell[0]][cell[1]] = tile
                value = self.maximizer(new_board, depth-1, alpha, beta)
                min_value = min(min_value, value)
                beta = min(beta, value)
                if beta <= alpha:
                    break  # Alpha cutoff
        return min_value

    def minimax_move(self, depth=4):
        best_move = None
        max_value = float('-inf')
        alpha = float('-inf')
        beta = float('inf')
        
        for direction in ["left", "right", "up", "down"]:
            new_board = self.perform_simulated_move(self.board, direction)
            if new_board != self.board:  # Skip invalid moves
                value = self.minimizer(new_board, depth-1, alpha, beta)
                if value > max_value:
                    max_value = value
                    best_move = direction
        return best_move

    # ------------------------- Game Controls -------------------------
    def perform_move(self, direction):
        new_board = self.perform_simulated_move(self.board, direction)
        if new_board != self.board:
            self.board = new_board
            self.add_new_tile()
            self.moves += 1
            self.update_gui()
            return True
        return False

    def key_pressed(self, event):
        if not self.game_over:
            moves = {
                "Left": "left", "Right": "right",
                "Up": "up", "Down": "down"
            }
            if event.keysym in moves:
                self.perform_move(moves[event.keysym])
                if self.check_game_over(self.board):
                    self.game_over = True
                    self.show_game_over()
    def has_2048(self):
        """Check if the board contains the 2048 tile."""
        return any(2048 in row for row in self.board)

    
    def auto_play_step(self):
        if self.game_over:
            return
    
        # Check if 2048 is already achieved
        if self.has_2048():
            self.game_over = True
            self.show_game_over(success=True)
            return
    
        direction = self.minimax_move()
        if not direction:
            self.game_over = True
            self.show_game_over()
            return
    
        self.perform_move(direction)
    
        # Check again after performing the move
        if self.has_2048():
            self.game_over = True
            self.show_game_over(success=True)
            return
    
        # Continue autoplay if game isn't over
        if not self.game_over:
            self.window.after(50, self.auto_play_step)

    def show_game_over(self, success=False):
        """Display game over/win message."""
        text = "2048 Achieved!" if success else "Game Over!"
        color = "#4CAF50" if success else "#8B0000"  # Green for success, red for failure
        label = tk.Label(self.window, text=text, font=("Helvetica", 24, "bold"),
                        bg="#BBADA0", fg=color)
        label.place(relx=0.5, rely=0.5, anchor="center")

    def start_autoplay(self):
        self.auto_playing = True
        self.auto_play_step()

if __name__ == "__main__":
    root = tk.Tk()
    game = G2049(root)
    root.mainloop()

## The best so far is 863 Moves for acheiving 2048
But as the game is random, there is no minimum number of moves at which we can be sure that guarentee that we will acheive 2048 after this. It is random. As given in the report this method guarentees the generation of the 2048 tile 71 times out of 100. Pretty Good Right?